In [6]:
from dbfread import DBF
import pandas as pd
import numpy as np

# Load punches_table
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)

# Extract columns from punches_table
token = [record['TOKEN'] for record in punches_table]
pdtime = [record['PDTIME'] for record in punches_table]
mode = [record['MODE'] for record in punches_table]

punches_df = pd.DataFrame({'token': token, 'pdtime': pdtime, 'mode': mode})

punches_df['pdtime'] = pd.to_datetime(punches_df['pdtime'], format='%d-%b-%y %H:%M:%S')

# Sort DataFrame by 'token', 'pdtime', and 'mode'
punches_df.sort_values(by=['token', 'pdtime', 'mode'], inplace=True)

# Create a new dataframe to store the calculated times
result_df = pd.DataFrame(columns=['token', 'date', 'in time', 'out time', 'total time'])

# Initialize variables to store in/out punch times
in_punch_time = None
out_punch_time = None

# Iterate over the rows and calculate time
for index, row in punches_df.iterrows():
    if row['mode'] == 0:
        in_punch_time = row['pdtime']
    elif row['mode'] == 1:
        out_punch_time = row['pdtime']
        
        result_df = pd.concat([result_df, pd.DataFrame({
            'token': [row['token']],
            'in time': [in_punch_time.strftime('%d-%b-%y %H:%M:%S')],
            'out time': [out_punch_time.strftime('%d-%b-%y %H:%M:%S')],
            'date': [pd.to_datetime(in_punch_time, format='%d-%b-%y %H:%M:%S').strftime('%d-%b-%y')]
        })], ignore_index=True)

# Sort the DataFrame by 'token', 'date'
result_df = result_df.sort_values(by=['token', 'date'])

# Save result_df to a CSV file
result_df.to_csv('./5thJan_punches.csv', index=False)

